In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import Layout, HTML
from IPython.core.interactiveshell import InteractiveShell
from ipyevents import Event
import threading

In [2]:
icons = ["️️🐇", "️️🦩", "️️🦢", "️️🦘", "🏠"]
friends = []
player = None
df = None
maze = None

def backgroundColor(cell):
    color = 'moccasin' if cell == "️️🐇" else 'white'
    return 'background-color: %s' % color

def initMaze(n):
    global df, maze, player
    df = pd.read_csv("./mazes/"+str(n)+".txt", header=None)
    m = df.to_numpy()
    m[m == "#"] = "️️🔲"
    for i in range(len(icons)):
        m[m == str(i)] = icons[i]
        
    l = np.where(m == icons[0])
    player = ({'icon':icons[0], 'location':(l[0][0], l[1][0])})
    
    maze = df.style.set_properties(**{'height': '50px', 'width':'50px', 'font-size':'50px', 'padding':'0px', 
                                      'margin':'0px', 'line-height':'1'}).applymap(backgroundColor)



In [3]:
def getCurrentFriendstatus():
    return "Rescued friends: " + ",".join(friends)

In [4]:
output = widgets.Output()
board = widgets.HBox([output], layout=Layout())

def refreshMaze():
    output.clear_output(wait=True)
    with output:
        status = HTML(getCurrentFriendstatus())
        display(maze, status)
        
def displayMaze():
    display(board)
    refreshMaze()

In [5]:
def move(key):
    global chars
    nl = None
    if key == "ArrowDown":
        nl = (player['location'][0]+1, player['location'][1])
    elif key == "ArrowLeft":
        nl = (player['location'][0], player['location'][1]-1)
    elif key == "ArrowRight":
        nl = (player['location'][0], player['location'][1]+1)
    elif key == "ArrowUp":
        nl = (player['location'][0]-1, player['location'][1])
    
    if df.iloc[nl] == '️️🔲' or df.iloc[nl] == '🏠':
        return
    if nl[0] >= 0 and nl[0] < df.shape[0] and nl[1] >= 0 and nl[1] < df.shape[1]:
        if df.iloc[nl] in icons and not df.iloc[nl] == icons[-1]:
            # Collect a friend
            if not df.iloc[nl] in friends:
                friends.append(df.iloc[nl])
        df.iloc[player['location'] ] = " "
        df.iloc[nl] = player['icon']
        player['location'] = nl

In [6]:
# Listen to keyboard events
boardEvent = Event(source=output, watched_events=['keydown'])
def handleEvent(event):
    if event['code'] in ['ArrowDown', 'ArrowUp', 'ArrowLeft', 'ArrowRight']:
        move(event['code'])
        refreshMaze()
boardEvent.on_dom_event(handleEvent)

In [7]:
def startGame():
    initMaze(1)
    displayMaze()

In [8]:
startGame()